# Exploration: Principal Component Analysis (PCA) and Receiver Operating Characteristic (ROC)

A fundamental component of mastering data science concepts is applying and practicing them. This exploratory notebook is designed to provide you with a semi-directed space to do just that with the Python, PCA, machine learning-based (ML-based) classification, and visualization skills that you either covered in an in-person workshop or through Microsoft Learn. The specific examples in this notebook apply Python and pandas concepts in a life-sciences context, but they are equally applicable across other disciplines and industry verticals.

This notebook is divided into different stages of exploration. Initial suggestions for exploration are more structured than later ones and can provide some additional concepts and skills for tackling data-science challenges with real-world data. However, this notebook is designed to provide you with a launchpad for your personal experimentation with data science, so feel free to add cells and run your own experiments beyond those suggested here. That is the power and the purpose of a platform like Jupyter Notebook!

## Setup and Refresher on Notebooks

Before we begin, you will need to import the principal libraries used to explore and manipulate data in Python: NumPy, pandas, and scikit-learn. The cell below also imports Matplotlib, the main visualization library in Python. For simplicity and consistency with prior instructions, industry-standard aliases are applied to these imported libraries. The cell below also runs the `%matplotlib inline` magic command, which instructs Jupyter to display Matplotlib output directly in the notebook. This cell also imports many of the specific functions from scikit-learn that you will need, but feel free to import others as you see fit in the course of your exploration.

In [ ]:
import pandas as pd
import numpy as np
from itertools import cycle
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

As it might have been a while since you last worked with Jupyter notebooks, here is a quick refresher on efficiently using them.

### Notebook Cells

Notebook cells are divided into Markdown text cells and interactive code cells. You can easily recognize code cells by the `[-]:` to the left of them.

Code in a code cells has only been executed -- and is thus available for use in other code cells in the notebook -- if there is a number beside the code cell (for example, `[1]:`).

To run the code in a cell, you can click the **Run** icon at the top left of a code cell or press **`Ctrl` + `Enter`**.

### Documentation and Help

Documentation for Python objects and functions is available directly in Jupyter notebooks. In order to access the documentation, simply put a question mark in front of the object or function in a code cell and execute the cell (for example, `?print`). A window containing the documentation will then open at the bottom of the notebook.

## Scenario

In this workshop, you will step into the role of a data scientist at a biotech firm. Specifically, you are tasked with making sense of experimental genetic data. The dataset is sparse and high-dimensional (64 observations across 3,116 features), so you will need to rely on principal component analysis (PCA) to conduct your initial examination of the data. Furthermore, your management is interested in both which features contribute to susceptibility to liver toxicity and which features contribute to insensitivity to it. Because multiple classes (sensitivity and insensitivity) will be involved in your classifications, you will also want to evaluate the performance of your initial ML investigation agaist each class (rather than just overall).

##  Dataset

The dataset that you have been asked to investigate comes from a liver toxicity study (Bushel *et al.*, 2007) in which 64 male rats were exposed to non-toxic (50 or 150 mg/kg), moderately toxic (1500 mg/kg), or severely toxic (2000 mg/kg) doses of acetaminophen (the active ingredient in Tylenol) in a controlled experiment. Necropsies were performed at 6, 18, 24, and 48 hours after exposure, and the mRNA from the liver was extracted. Ten clinical chemistry measurements of variables containing markers for liver injury are available for each subject, and the serum enzymes levels are measured numerically. (See the [dataset description](https://www.rdocumentation.org/packages/mixOmics/versions/6.3.2/topics/liver.toxicity) for more details.)

The data were further normalized and pre-processed by Bushel *et al.* (2007) and are available to you in four files:
 - **gene.csv**: The expression measure of 3116 genes for the 64 subjects (rats)
 - **clinic.csv**: The 10 clinical variables for the same 64 subjects
 - **treatment.csv**: The treatment information on the 64 subjects, such as doses of acetaminophen and times of necropsies
 - **gene.ID**: GeneBank IDs and gene titles of the annotated genes

### Reference
Bushel, P., Wolfinger, R. D. and Gibson, G. (2007). Simultaneous clustering of gene expression data with clinical chemistry and pathological evaluations reveals phenotypic prototypes. *BMC Systems Biology* **1**, Number 15.

## Section 1: Guided Exploration

The first task in any data role is easy: import the data into a DataFrame and quickly look over it to see what you are dealing with.

In [ ]:
# Import the data from genes.csv into a DataFrame.


The dataset may only have 64 observations (one for each rat), but each one has 3,116 features (each gene). To fully visualize this dataset requires a 3,116-dimensional space (one dimension per feature/gene), an impossibilty to deal with directly. Looking at just a subset of the features can also be tricky: how will researchers know if they selected the right features? This is why you have been tasked with performing PCA on it: correctly done, PCA can strike a balance of projecting the dataset down into a low-enough dimensional space that visualization is feasible and also preserving necessary information in the process. 

### Run PCA

To begin the process, fit a PCA model for the data and then graph the variance of the PCA components (the `explained_variance_ratio_` attribute of the PCA model), looking for the elbow in that graph (as you did in the Reactors module on PCA). To do so, you will need to create a scikit-learn PCA oject and then use its [`fit_transform()`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA.fit_transform) method. This will fit the model used to performn the dimensionality-reduction transformation on the data.

In [ ]:
# Create a PCA object and use its fit_transform() method.


We are looking for the 'elbow' in the graph: the point of inflection at which we start to get far less explained variance for each additional component.

The elbow appears to be at about 4. Check the sum of the variance of the first four PCA components, and then print out the sum and the explained variance.

In [ ]:
# Print out the sum of the explained variance of the first four components.


In [ ]:
# Print out the explained variance of the first four components.


**Group or Partner Discussion**

 - What does this array mean? Why should you care about these numbers and what do they tell you in the context of this liver toxicity dataset?

### Create the PCA DataFrame

As you did in the Reactors PCA module, create a DataFrame from the first four PCA components (the ones that encompass about 70% of your data's variance). You will need this DataFrame in order to begin visually exploring the "flattened" data (that is, of the dataset projected down from 3,116 dimension down to four).

In [ ]:
# Create a new DataFrame consisting of the first four components.


For convenience and clarity, rename the columns of the DataFrame 'c1' through 'c4'. (You may find the [`pandas.DataFrame.rename`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rename.html) method helpful for this.)

**Group or Partner Discussion**

 - Review the Reactors PCA module. What do the numbers in this DataFrame represent?


In [ ]:
# Rename the columns of the DataFrame 'c1' through 'c4'.


### Import the treatement data

You currently have the gene data but nothing to predict or classify from it. With just the gene data, we don't actually know anything about the connection between a particular gene signature and the results of treatment. In order to start discovering connections between the genes and the treatment regimens, import the data from `treatment.csv` into a new DataFrame.

In [ ]:
# Import the treatment.csv data into a new DataFrame.


Finally, join the columns of the treatment DataFrame in which we are interested (`Dose.Group` and `Time.Group`) to the PCA DataFrame to prepare for visualization; the other columns are either subject ID numbers or a restatement of treatment dosage and won't help us in our visualization. (That said, changing the DataFrame index to be the test-subject ID number can make the DataFrame easier to read for colleagues and researchers; the [`pandas.DataFrame.set_index`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.set_index.html) method can help with that.)

In [ ]:
# Join the Dose.Group and Time.Group columns to your PCA DataFrame.
# You can also make your DataFrame more usable to colleagues by making the DataFrame index the test-subject ID number.


## Section 2: Intermediate Exploration

It is now time to begin the meaningful part of PCA: looking for patterns in the lower-dimensional projection of the data. To do this, you will need to create a visualization.

In previous workshops, we have looked at scatterplots as an important means of visualizing multivariate numeric data, which is exactly what your PCA eigenvectors are. Being your visual inspection of the reduced data by creating a 2D scatterplot of the first two columns of your PCA DataFrame (these should correspond with $c_{1}$ and $c_{2}$, which together account for over half of the variance in the dataset). Add some context to the graph by making the colors for the points on the scatterplot correspond to the dosage received by the test subjects (the rats); doing so will help you see patterns within the data.

This [tutorial](https://pythonspot.com/3d-scatterplot/) on creating 3D scatterplots can be adapted for creating your 2D scatterplot. You might also find the `matplotlib.pyplot.scatter` [documentation](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.scatter.html#matplotlib.pyplot.scatter) helpful.

In [ ]:
# Create a scatterplot of the first two components, color-coded by dosage.


The purpose of this scatterplot is to help us assess whether there are any patterns in the data in just the first two components--which there are! The 50-mg and 150-mg dosages form a nice cluster with a very distinct gap between it and the looser cluster of higher dosages. This would indicate that there are genes that correlate to lethal liver toxicity at lower dosages of acetaminophen.

Does this same patten hold for the time until death? To see whether it does, create the same kind of scatterplot as the last one, this time letting the colors encode the amount of time until necropsies of the subjects were conducted.

In [ ]:
# Create a scatterplot of the first two components, color-coded by time until necropsy.


**Group or Partner Discussion**

 - Did it surprise you that the scatterplots for dosage and time are the same, except with the dots having different colors? Why should this be the case?
 - Did the addition of color-coding for dosage or time to necropsy 'add back' any of the variance in the dataset lost by projecting it down into two dimensions? If not, did the color-coding nevertheless add useful information to the plot? If so, what information did you gain from the addition of the color-coding?

Now let's consider the visualization in three dimensions. Alter your scatterplot code now, and also incorporate the third column from the PCA DataFrame (`c3`). You can color-code for either dosage or time, although dosage might present a clearer picture. (You might find the `mpl_toolkits.mplot3d.axes3d.Axes3D` [documentation](https://matplotlib.org/3.2.1/api/_as_gen/mpl_toolkits.mplot3d.axes3d.Axes3D.html) to be helpful.)

In [ ]:
# Create a 3D scatterplot of the first three components color-coded by dosage.


**Note:** Try a variety of settings for the `elev=` and `azim=` parameters in [`Axes3D`](https://matplotlib.org/3.2.1/api/_as_gen/mpl_toolkits.mplot3d.axes3d.Axes3D.html) until you find some that help you see patterns in the data.

**Group or Partner Discussion**

 - What patterns do you see in the data? Although adding $c_{3}$ to the plot raises the total amount of variance from the original dataset up to close to 63%, did doing so help you better discern patterns in the data? If so, how? If not, why not?

You can also add more information to the plot without adding additional dimensions. For example, alter your 3D plot to keep dosages color-coded, but make the dots different sizes based on the time to necropsy. (Do this by making the value for the `s=` parameter equal to the `Time.Group` column in your PCA DataFrame. Note that the differences in times is not great, which means that you might want to multiply this value by a constant or raise the time values to a power in order to make the sizes in dots more distinct.)

In [ ]:
# Create a 3D scatterplot of the first three components color-coded by dosage.
# Make the marker size proportional to time until necropsy.


We now have a visualization with five aspects: the three principal components of the dataset, the dosage for the test subjects (the colors of the markers), and the time to necropsy for the subjects (the size of the markers). The now-familiar gap between the cluster of low dosages and the high dosages is still apparent, but adding the time to necropsy does not seem to add much more useful information: no clear patterns among it appear in this plot.

The principal question to ask with all visualizations is: "Does it help my data speak more clearly?" In the case of your role as data investigator, does the third dimension better elucidate patterns in your data? Possibly, possibly not. Let's look at an alternative to a 3D scatterplot: plotting the data across four 2D scatterplots (one for each time value). To do so, return to your 2D scatterplot code and nest your original `for` loop inside another `for` loop that iterates over each of the unique values in the `Time.Group` column in the PCA DataFrame.

(**Note:** You might need to increase the size of the `figsize=` parameter for [`matplotlib.pyplot.figure`](https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.figure.html) in order to clearly see all of the plots.)

In [ ]:
# Create a 2x2 set of scatterplots of the first two components by time and dosage.


Splitting the plot across four subplots let's us clearly see the relationships of dosages between the two principal components for each time group; being plotted in only two dimensions, we do lose information about how these points lie in the third principal component (the z-axis in our previous plots), but that component only accounts for 9% of total variance for the dataset, so we do not lose very much information. The cluster of low dosages is still clearly visible in the lower-left corner of each subplot, but the data points for higher dosages are mingled in each of the subplots, so no additional patterns emerge for those. We may have gleaned about as much information as we can from visualization. 

## Section 3: Individual Exploration

Visual inspection of the data is good, but at some point we want to introduce ML into our investigations. In the Reactors ROC module, you learned about the receiver operating characteristic (ROC) curve as a means to evaluate the performance of ML models. Because the performance of most classification algorithms is a tradeoff between false-positive and false-negative rates, the ROC curve provides a graphical illustration of the ratio of an algorithm's true-positive rate to its false-positive rate. In this section, you will use the ROC curve of a classification model to learn more about the dataset you have been tasked with investigating in your assumed role.

Normally, ROC curves only work for binary models. After all, the curve is the plot of the true positive rate against the false positive rate at various thresholds. However, using the [One versus Rest](https://en.wikipedia.org/wiki/Multiclass_classification) technique, you can use binary classification algorithms for multi-class classification and then plot multiple ROC curves on top of one another.

To do this, adapt this code from the scikit Learn documentation site to work with your liver-toxicity dataset: https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#plot-roc-curves-for-the-multilabel-problem

(**Tip:** Classify the data with dosage being your response; use colors for the ROC curves that match the colors you used in your scatterplots to make interpretation easier.)

In [ ]:
# Select all of the columns except the first from the genes DataFrame for your predictors.
# Select the dosage column of the treatment DataFrame as your response.


You ROCs curves should reflect a mixture of superb performance to abysmal output for the classification algorithm. On the good end of the spectrum--class 0, which corresponds to 50 mg dosage group--we have perfect classification: a 100% true positive rate with no false positives. On the other end of the performance spectrum, class 2 (the 1500 mg dosage group) provides several instances in which it supplies more false positives than true ones (any time the ROC curve dips below the diagonal dotted line on the plot).

A related metric shown in this plot is the area under the curve. This is just what it sounds like: a measure of the area of the region bounded by the ROC curve and the bottom of the plot. What does this metric represent? If we look at our perfect classifier (class 0), it has an area under curve (AUC) of 1.00, whereas the rather poor class 2 has an AUC just over 0.50 (representing, on average, a classifier that supplies as many false positives as true ones). A higher AUC is an indicator of better overall classifier performance.

The macro-avarage and micro-average curves are two ways of assessing overall performance of all of the classifiers. The macro-average is a simple average of the performance of each classifier for each false-positive rate; the micro-average is a weighted average of these. See [this response](https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin) on Stack Exchange for a more complete explanation.

**Group or Partner Discussion**

 - What does this range of performance by the classifier tell you about the dataset, if anything?

Note that code you repurposed used a classification algorithm called *support vector machines* (SVMs), which we have not yet addressed in the Reactors modules. Generate the multi-class ROC curves again, but this time use a classification algorithm you are familiar with: logistic regression. Not only is logistic regression a classification algorithm that we have examined before in the Reactors modules, it will lend itself to clearer interpretation when you will be tasked later on with tying results back to individual genes.

In addition, run this with `Time.Group` as the response rather than `Dose.Group`, which will also prove helpful later on.

In [ ]:
# Same predictors as last time but with Dose.Group for the response.


You should notice a substantial improvement to classification performance using logistic regression for all classes. If you are curious, remember that you can check the overall accuracy of your model by using `metrics.accuracy_score` from `sklearn`.

In [ ]:
# Use a one-versus-rest logistic-regression classifier.


In [ ]:
# Use a one-versus-rest SVM classifier.


There are two take-aways from this. First, while logistic regression did provide better performance on its own, it was nowhere near dramatic enough to explain the massive improvements we saw in the ROC curves generated with that classifier; the change in the response from `Dose.Group` to `Time.Group` supplied much of that. (If you look at this with logistic regression on `Dose.Group`, the performance will not be much different than with SVMs.)

Second, different algorithms will perform differently with a different set of predictors and responses. For this reason, you should always use a variety of algorithms to see which will work best with your data **and** the specific questions that you are asking about your data.

So far we've noted that while distinguishing between the 1,500 mg/kg and 2,000 mg/kg dose-groups was problematic, our classifier was quite accurate in distinguishing between the low- and high-dosage groups. We also saw that classifying by time-to-necropsy was quite accurate. With this in mind, it ought to be possible to identify which genes contribute most to rats that die quickly from low doses of acetaminophen and which ones that contribute to rats that die slowly even from high doses.

As a final task for this module, pull some tangible insights from the dataset. To identify those genes, create a new column in the treatment DataFrame that flags whether a dosage is not toxic (at or below 150 mg/kg) or toxic. Use that column to create another column, this one dividing the observations into eight categories (short time to death at non-toxic dosage, short time to death at toxic dosage, etc.). You might find the `numpy.where` function and the `pandas.Series.map` method useful in doing this.

Once you have those eight classes in place, run one-versus-rest logistic regression with those new classes as the response, and create multi-class ROC curves to evaluate the classifier.

In [ ]:
# Create a new binary column in the treatment DataFrame for dosages that are and are not toxic.


In [ ]:
# Create a list of unique times from the Time.Group column.

# Create a dictionary of even numbers for each of these unique times.
# You will use this dictionary as the basis of a new column in the DataFrame.
# This column will help group together rats that died quickly or slowly from toxic and non-toxic dosages.


In [ ]:
# Create a new DataFrame column that creates these groups of susceptible and resistant rats for each time group.
# Do so by adding the value of the Toxic column to the entries in time_dict for each time.


In [ ]:
# Use the new Group column as the response for the classifier.


The classification should perform well on the classes of interest to us: the first class (class 0: low-dosage, quick death) and the last class (class 7: high-dosage, slow death).

Identifying the genes most responsible for this behavior requires thinking about the how logistic regression works. Recall that the equation for the logistic regression curve interest is
$$
y = \frac{1}{1 + e^{-(β_0 + β_1X_1 + β_2X_2 + β_3X_3 + \cdots + β_nX_n)}}
$$
where a response less than 0.5 classifies an observation as negative (not in the class in question) and a response greater than 0.5 classifies it as positive (in the class in question) in a binary classification. Because we are dealing with a negative exponent in the denominator of a fraction, positive coefficients (the $β$s) will tend to make an observation get closer to 1 (positive), and negative coefficients will trend it toward 0 (negative). We can thus make the argument that genes with exceptionally large-magnitude positive coefficients carry more weight with classifying an observation into the our desired class.

Your final task is to access the `coef_` attribute for the fitted logistic regression model and print out those genes that correspond to coefficients above an arbitrary threshold (such as 0.15).

In [ ]:
# Print out all of the genes whose first coefficient is greater than 0.15.


In [ ]:
# Print out all of the genes whose last coefficient is greater than 0.15.


Your printed lists should be rather short. Note that we can't say for certain that these genes specifically grant rats either vulnerability to or protection from liver toxicity due to ingesting acetaminophen. Rather, we have trimmed down the list of possible genes from 3,116 to a handful that scientists can start to investigate in greater detail. But in your assumed role as a data scientist, you have made the research problem for the other scientists on your team a much more manageable one.